# CHEME 5660: Introduction to Delta Hedging for American Option Contracts

## Introduction
Suppose we have a composite trade consisting of a combination of risky assets, i.e., shares of `XYZ`, and options contracts on `XYZ`. Let the assets for this trade be described by the set $\mathcal{A}$. Then, the overall $\Delta$ of the composite trade for asset set $\mathcal{A}$ is given by:

$$\Delta = \sum_{a\in\mathcal{A}}\theta_{a}n_{a}\hat{\Delta}_{a}$$

where $n_{a}$ denotes the copy number of asset $a$, i.e., the number of contracts or shares of asset $a$ in the portfolio, $\hat{\Delta}_{a}$ denotes the delta for asset $a$ and $\theta_{a}$ denotes a direction parameter; $\theta_{a} = 1$ if we purchased asset $a$, while $\theta_{a} = -1$ if we sold asset $a$. A riskless (delta-hedged) portfolio with respect to the share price of `XYZ` satisfies the condition:

$$\Delta = 0$$

The $\hat{\Delta}_{a}$ values for different asset classes.

* If asset $a$ is an option contract, then $\hat{\Delta}_{a}$ takes the value $\hat{\Delta}_{a} = 100\cdot\Delta_{a}$ where $\Delta_{a}$ is the delta value for a long position in option contract $a$. 
* If asset $a$ are shares of stock, then $\hat{\Delta}_{a}$ is given by $\hat{\Delta}_{a} = 1$

### Problem
Let's explore setting up a $\Delta$-hedge on a short-strangle position on `NFLX`. We'll set up the hedge using values for `10/25/22` and look at how the hedged position performs today `10/26/22`.

## Lab 6 Setup
The code block below installs (and loads) any [Julia](https://julialang.org) packages that we need to complete the calculations.

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-6-Dynamic-Delta-Hedging`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-6-Dynamic-Delta-Hedging/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-6-Dynamic-Delta-Hedging/Manifest.toml`


In [2]:
# packages that we need -
using PQEcolaPoint
using DataFrames
using CSV
using Statistics
using Dates
using Plots
using Colors
using Distributions
using PrettyTables

# paths -
const _ROOT = pwd();
const _PATH_TO_DATA = joinpath(_ROOT,"data");
const _PATH_TO_FIGS = joinpath(_ROOT,"figs");

### Load the lab 6 code library
The call to the `include` function loads the `CHEME-5660-Lab-6-Library.jl` library into the notebook; the library contains types and functions we use during the lab:

__Functions__:
* The function `ticker(type::String, underlying::String, expiration::Date, K::Float64)-->String` produces the [option ticker symbol](https://polygon.io/blog/how-to-read-a-stock-options-ticker/).

In [3]:
include("CHEME-5660-Lab-6-Library.jl");

## Sample calculation: $\Delta$, $\Theta$ and $\mathcal{V}$ for Netflix Call and Put Option Contracts

In [4]:
# setup constants
DTE = 52.0         # number of days to expiration (assumes all options purchased on 10/25/22)
μ̂ = 0.0423         # risk free rate 
L = 100            # number of levels in the tree
B = 365.0;         # number of days in a year
Sₒ = 289.26;       # initial share price 

### Step 1: Build contract models

In [5]:
# Build the contracts (Call = 1, Put = 2)

# Expiration dates -
D₁ = Date(2022, 12, 16);
D₂ = Date(2022, 12, 16);

# Strike prices -
K₁ = 315.0 # units: USD/share
K₂ = 255.0 # units: USD/share

# Build the tikers -
T₁ = ticker("C","NFLX",D₁,K₁);
T₂ = ticker("C","NFLX",D₂,K₂);

# IV -
IV₁ = (48.72/100.0);
IV₂ = (55.21/100.0);

# build the contracts 
netflix_call_contract_model = build(CallContractModel, (
    ticker = T₁,
    expiration_date = D₁,
    strike_price = K₁,
    premium = 0.0,
    number_of_contracts = 1,
    direction = 1, # Bug or feature? => always use 1 here (even if we are selling)
    current_price = 0.0      
));

netflix_put_contract_model = build(PutContractModel, (
    ticker = T₂,
    expiration_date = D₂,
    strike_price = K₂,
    premium = 0.0,
    number_of_contracts = 1,
    direction = 1, # Bug or feature? => always use 1 here (even if we are selling)
    current_price = 0.0      
));

### Step 2: Compute the Greeks for a Put and Call contract

In [6]:
# Delta Δ -
Δ₁ = δ(netflix_call_contract_model; Sₒ= Sₒ, number_of_levels=(L+1), σ=IV₁, T=(DTE/B), μ=μ̂);
Δ₂ = δ(netflix_put_contract_model; Sₒ= Sₒ, number_of_levels=(L+1), σ=IV₂, T=(DTE/B), μ=μ̂);

In [7]:
# Theta Θ -
Θ₁ = θ(netflix_call_contract_model; Sₒ= Sₒ, number_of_levels=(L+1), σ=IV₁, T=(DTE/B), μ=μ̂);
Θ₂ = θ(netflix_put_contract_model; Sₒ= Sₒ, number_of_levels=(L+1), σ=IV₂, T=(DTE/B), μ=μ̂);

In [8]:
# Vega V -
vega₁ = vega(netflix_call_contract_model; Sₒ= Sₒ, number_of_levels=(L+1), σ=IV₁, T=(DTE/B), μ=μ̂);
vega₂ = vega(netflix_put_contract_model; Sₒ= Sₒ, number_of_levels=(L+1), σ=IV₂, T=(DTE/B), μ=μ̂);

### Step 3: Make a Table

In [23]:
# Build a pretty table to display our greek values
sample_greek_table_data = Array{Any,2}(undef, 3, 3);

# rows: 1 = Delta, 2 = Theta, 3 = Vega
# cols: 1 = Label, 2 = Call, 3 = Put

# Delta -
sample_greek_table_data[1,1] = "Δ"
sample_greek_table_data[1,2] = Δ₁
sample_greek_table_data[1,3] = Δ₂

# Theta -
sample_greek_table_data[2,1] = "Θ"
sample_greek_table_data[2,2] = Θ₁
sample_greek_table_data[2,3] = Θ₂

# Vega -
sample_greek_table_data[3,1] = "Vega"
sample_greek_table_data[3,2] = vega₁
sample_greek_table_data[3,3] = vega₂

# header -
sample_greek_table_header = (["Greek", "NFLX Call K = $(K₁) (long)", "NFLX Put K = $(K₂) (long)"])

# display -
pretty_table(sample_greek_table_data; header = sample_greek_table_header)

┌───────┬────────────────────────────┬───────────────────────────┐
│ Greek │ NFLX Call K = 315.0 (long) │ NFLX Put K = 255.0 (long) │
├───────┼────────────────────────────┼───────────────────────────┤
│     Δ │                   0.353717 │                 -0.204809 │
│     Θ │                  -0.200519 │                 -0.159098 │
│  Vega │                   0.402955 │                  0.325732 │
└───────┴────────────────────────────┴───────────────────────────┘


## Excercises

### a) Compute the $\Delta$ of a Short Strangle on Netflix `NFLX`
Let the current share price of `NFLX` be $S_{\circ}$. A short strangle position on `NFLX` can be constructed by selling two option contracts:
* Sell one out of the money `Call` contract ($K_{1}>S_{\circ}$)
* Sell one out of the money `Put` contract ($K_{2} < S_{\circ}$).

Let's assume we sold the 315-Call and 255-Put on `NFLX` (with expirations on 12/16/22) at the close on 10/25/22. Thus, we can use the Greek data from the sample calculation.

In [10]:
# Compute the Δ of the position *before* the open on 10/26 -
Δ = -1*100*Δ₁ + -1*100*Δ₂

-14.890711669553092

In [11]:
# How many shares do we need to buy (or sell) to Δ-hedge this positon?
n₃ = -Δ

14.890711669553092

### b) Compute the P/L of the hedged versus a non-hedged portfolio for $\epsilon$ change in `NFLX` share price

##### Step 1: Recompute the option prices for the current market conditions

In [12]:
# Orignal option prices -
P₁ = 12.70 # Call mark price (original price)
P₂ = 8.50  # Put mark price (original price)

# Update the share price -
S_now = 303.60
ϵ = (S_now - Sₒ)/Sₒ;  # change in price per share *after* market open on 10/26

# what is the change in price?
ΔS = Sₒ*ϵ;

# *Estimate* the new option prices *after* market open on 10/26
C_now = P₁ + Δ₁*ΔS;
P_now = P₂ + Δ₂*ΔS;

##### Case 1: Without the hedge (now)
What is the net profit (or loss) per share if we closed the `NFLX` short strangle now _without_ the $\Delta$-hedge?

In [13]:
PL = (P₁ - C_now) + (P₂ - P_now) # per share profit/loss

-2.135328053413918

##### Case 2: With the hedge (now)
What is the net profit (or loss) per share if we closed the `NFLX` short strangle now _with_ the $\Delta$-hedge?. 
* We purchased $n_{3}$ shares of `NFLX` at the close on `10/25/2022` at a price of $S_{close} = S_{\circ}$ USD/share.
* What happens if we can't get the shares unil the open of `10/26/22`?

Now, the `NFLX` price is $S_{now}$ USD/share. 

In [15]:
# Netflix price -
S_close = Sₒ; # NFLX close price 10/25/22
S_open = 293.54; # NFLX open price 10/26/22

S_hedge = S_open
PL_hedge = 100*(PL) + n₃*(S_now - S_hedge)

-63.73224594568765

### c) How good are our estimates (should we re-hedge?)
Let's compare the estimated versus calculated prices for the options, and the $\Delta$-values. Live values taken at `12 PM 10/26/22`:

In [21]:
# initialize -
check_table_data = Array{Any,2}(undef, 2, 5);

# values -
sense_value = -1

# Call -
check_table_data[1,1] = "Call"
check_table_data[1,2] = Δ₁
check_table_data[1,3] = 0.4650
check_table_data[1,4] = C_now
check_table_data[1,5] = 17.93

# Put -
check_table_data[2,1] = "Put"
check_table_data[2,2] = Δ₂
check_table_data[2,3] = -0.1659
check_table_data[2,4] = P_now
check_table_data[2,5] = 6.23

# table header -
check_table_header = (["Contract", "Delta","Delta (actual)","Price", "Price (actual)"]);

# show -
pretty_table(check_table_data; header=check_table_header)

┌──────────┬───────────┬────────────────┬─────────┬────────────────┐
│ Contract │     Delta │ Delta (actual) │   Price │ Price (actual) │
├──────────┼───────────┼────────────────┼─────────┼────────────────┤
│     Call │  0.353717 │          0.465 │ 17.7723 │          17.93 │
│      Put │ -0.204809 │        -0.1659 │ 5.56303 │           6.23 │
└──────────┴───────────┴────────────────┴─────────┴────────────────┘


### Additional Resources
* [Hans Bühler, Lukas Gonon, Josef Teichmann, Ben Wood (2018). Deep Hedging. arXiv:1802.03042,
DOI: https://doi.org/10.48550/arXiv.1802.03042](https://arxiv.org/abs/1802.03042)